In [246]:
from facebook_business.api import FacebookAdsApi
from facebook_business import FacebookSession
from facebook_business.adobjects.user import User
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adpreview import AdPreview
# from facebookads.adobjects.adset import AdSet
# from facebookads.adobjects.adimage import AdImage
# from facebookads.adobjects.adspixel import AdsPixel
from facebook_business.adobjects.adcreative import AdCreative
from facebook_business.adobjects.ad import Ad
# from facebookads.adobjects.targeting import Targeting
import json
import os
import unicodedata

In [247]:
# cool AD data
my_app_id = "2280157165384459"
my_app_secret = "5caa8b5c49cf78f2285f9921b51baf98"
my_access_token_1 = "EAAgZAynXf8wsBAECSf1NPj4ZAajFgeagcXWj5cZAa82MkOIGFOqzNe4pkhX5ZCzhBeNwO2gU32qvOamvu1QXCeCjWnIKoXb2ry9guneMZB49789ZBPSClGcRlZBZASmIJfRKYHtV7VGJygUS5sowDOGzP2n25OW94dUZD"

session1 = FacebookSession(
    my_app_id,
    my_app_secret,
    my_access_token_1
)

api1 = FacebookAdsApi(session1)
# 2039741129633793
print(api1)

FacebookAdsApi.set_default_api(api1)
me = User(fbid='me', api=api1)
my_accounts = list(me.get_ad_accounts())

my_account = my_accounts[0]

my_account_id = 'act_2039741129633793'

account = AdAccount(my_account_id)

In [248]:
# last_3_months
# this_month
# last_week
# this_week
# lifetime
# https://developers.facebook.com/docs/marketing-api/insights/parameters/v2.7
time_range = "last_week"

In [249]:
# EXTRACT
# ad insights data
insights = account.get_insights(fields=[
    'impressions',
    'ad_id',
    'spend',
    'unique_clicks',
    'unique_actions',
    'cpm',
    'cpc',
    'adset_name',
    'ad_name',
    'campaign_name',
    'actions',
    'cpp',
    'ctr',
    'reach',
    'unique_clicks',
    'date_start',
    'date_stop'
], params={
    'level': 'ad',
    'date_presets': time_range
})

In [250]:
# TRANSFORM
# only include ad if it has clicks and actions
import pandas as pd
ad_insights = []
for ad in insights:
    if 'cpc' in ad:
        cool_ad = {
            "ad_id": ad['ad_id'],
            "ad_name": ad['ad_name'],
            "adset_name": ad['adset_name'],
            "campaign_name": ad['campaign_name'],
            "clicks": int(ad['unique_clicks']),
            "spend": float(ad['spend']),
            "cpc": float(ad['cpc']),
            "cpm": float(ad['cpm']),
            "date_start": ad['date_start'],
            "date_stop": ad['date_stop'],
            'impressions': int(ad['impressions']) 
        }
        try:
            actions = ad['unique_actions']
            cart_adds = list(filter(lambda action: action['action_type'] == 'offsite_conversion.fb_pixel_add_to_cart', actions))
            checkouts = list(filter(lambda action: action['action_type'] == 'offsite_conversion.fb_pixel_initiate_checkout', actions))
            view_contents = list(filter(lambda action: action['action_type'] == 'offsite_conversion.fb_pixel_view_content', actions))
            purchases = list(filter(lambda action: action['action_type'] == 'offsite_conversion.fb_pixel_purchase', actions))
            cool_ad['cart_adds'] = 0 
            cool_ad['checkouts'] = 0
            cool_ad['view_contents'] = 0
            cool_ad['purchases'] = 0
            if len(cart_adds) != 0: cool_ad['card_adds'] = int(cart_adds[0]['value'])
            if len(checkouts) != 0: cool_ad['checkouts'] = int(checkouts[0]['value'])
            if len(view_contents) != 0: cool_ad['view_contents'] = int(view_contents[0]['value'])
            if len(purchases) != 0: cool_ad['purchases'] = int(purchases[0]['value'])
            ad_insights.append(cool_ad)
        except:
            print('ADDDDD')
            print(ad)
    
ad_insights_df = pd.DataFrame(ad_insights)

ADDDDD
<AdsInsights> {
    "ad_id": "23843190453880483",
    "ad_name": "Suit Collection_LLA_IG Only_Image",
    "adset_name": "LLA 3% US Initiate Checkout 180 IG - Website",
    "campaign_name": "CM_WC_2019",
    "cpc": "0.86",
    "cpm": "5.308642",
    "cpp": "5.308642",
    "ctr": "0.617284",
    "date_start": "2019-01-25",
    "date_stop": "2019-02-23",
    "impressions": "162",
    "reach": "162",
    "spend": "0.86",
    "unique_clicks": "1"
}


In [251]:
len(ad_insights)

143

In [252]:
ad_insights_df = pd.DataFrame(ad_insights)

In [253]:
len(ad_insights_df)

143

In [254]:
ad_insights_df.columns.values

array(['ad_id', 'ad_name', 'adset_name', 'campaign_name', 'card_adds',
       'cart_adds', 'checkouts', 'clicks', 'cpc', 'cpm', 'date_start',
       'date_stop', 'impressions', 'purchases', 'spend', 'view_contents'],
      dtype=object)

In [255]:
# EXTRACT
# creative data
creatives = account.get_ad_creatives(fields=[
    'image_url',
    'title',
    "object_story_spec"
], params={
    'level': 'ad',
    'date_presets': time_range
})

In [256]:
ad_creatives = []
for creative in creatives:
    title = ''
    image_url = ''
    object_story_spec = {}
    
    if 'title' in creative:
        title = creative['title']
        
    if 'image_url' in creative:
        image_url = creative['image_url']
        
    if 'object_story_spec' in creative:
        object_story_spec = creative['object_story_spec']
        
    ad_creatives.append({
        'id': creative['id'],
        'object_story_spec': object_story_spec,
        'image_url': image_url,
        'title': title
    })

In [257]:
len(ad_creatives)

1228

In [258]:
ad_creatives_df = pd.DataFrame(ad_creatives)

In [259]:
len(ad_creatives_df[ad_creatives_df.title == ""])

375

In [260]:
def extract_title(ad):
    title, object_story_spec = ad
    
    if title != "":
        return title
    else:
        if 'link_data' in object_story_spec and 'name' in object_story_spec['link_data']:
            return object_story_spec['link_data']['name']
        elif 'link_data' in object_story_spec and "child_attachments" in object_story_spec['link_data']:
            return 'carousel'
        else:
            return ""
        
ad_creatives_df['title'] = ad_creatives_df[['title', 'object_story_spec']].apply(extract_title, axis=1)

In [261]:
len(ad_creatives_df[ad_creatives_df.title == ""])

245

In [264]:
# EXTRACT 
# ads
ads = account.get_ads(fields=[
    'targeting',
    'id',
    'bid_amount',
    'bid_type',
    'status',
    'creative'
], params={
    'level': 'ad',
    'date_presets': time_range
})

In [265]:
# extract ads 
ad_targeting = []
for ad in ads:
    try:
        ad_targeting.append({
            'id': ad['id'],
            'targeting': ad['targeting'],
            'status': ad['status'],
            'creative': ad['creative']['id'],
        })
    except:
        print(ad)

In [266]:
len(ad_targeting)

766

In [267]:
ads_df = pd.DataFrame(ad_targeting)

In [268]:
len(ads_df)

766

In [269]:
def extract_min_age(targeting):
    return targeting['age_min']

def extract_max_age(targeting):
    return targeting['age_max']

def is_mobile(targeting):
    try:
        if 'mobile' in targeting['device_platforms']:
            return 1
        else:
            return 0
        
    except:
        print(targeting)
    
def is_female(targeting):
    if 'genders' in targeting and 2 in targeting['genders']:
        return 1
    else:
        return 0

def is_male(targeting):
    if 'genders' in targeting and 1 in targeting['genders']:
        return 1
    else:
        return 0
    
def is_facebook(targeting):
    if 'publisher_platforms' in targeting and "facebook" in targeting['publisher_platforms']:
        return 1
    else:
        return 0
    
def is_instagram(targeting):
    if 'publisher_platforms' in targeting and "instagram" in targeting['publisher_platforms']:
        return 1
    else:
        return 0
    
def is_messenger(targeting):
    if 'publisher_platforms' in targeting and "messenger" in targeting['publisher_platforms']:
        return 1
    else:
        return 0
        
def is_audience_network(targeting):
    if 'publisher_platforms' in targeting and "audience_network" in targeting['publisher_platforms']:
        return 1
    else:
        return 0 

df['min_age'] = df.targeting.apply(extract_min_age)
df['max_age'] = df.targeting.apply(extract_max_age)
df['is_female'] = df.targeting.apply(is_female)
df['is_male'] = df.targeting.apply(is_male)
df['is_facebook'] = df.targeting.apply(is_facebook)
df['is_instagram'] = df.targeting.apply(is_instagram)
df['is_messenger'] = df.targeting.apply(is_messenger)
df['is_audience_network'] = df.targeting.apply(is_audience_network)

In [270]:
df.is_audience_network.sum()

14

In [271]:
# tie them all together and send an email to the gang with CSV 
# publish top ad to Slack using mode
print(ad_insights_df.columns.values)
print(ads_df.columns.values)

['ad_id' 'ad_name' 'adset_name' 'campaign_name' 'card_adds' 'cart_adds'
 'checkouts' 'clicks' 'cpc' 'cpm' 'date_start' 'date_stop' 'impressions'
 'purchases' 'spend' 'view_contents']
['creative' 'id' 'status' 'targeting']


In [272]:
df = ad_insights_df.join(ads_df, lsuffix='ad_id', rsuffix='id')

In [273]:
df.columns.values

array(['ad_id', 'ad_name', 'adset_name', 'campaign_name', 'card_adds',
       'cart_adds', 'checkouts', 'clicks', 'cpc', 'cpm', 'date_start',
       'date_stop', 'impressions', 'purchases', 'spend', 'view_contents',
       'creative', 'id', 'status', 'targeting'], dtype=object)

In [274]:
len(df)

143

In [275]:
len(ad_insights_df)

143

In [276]:
len(ads_df)

766

In [277]:
df.columns.values

array(['ad_id', 'ad_name', 'adset_name', 'campaign_name', 'card_adds',
       'cart_adds', 'checkouts', 'clicks', 'cpc', 'cpm', 'date_start',
       'date_stop', 'impressions', 'purchases', 'spend', 'view_contents',
       'creative', 'id', 'status', 'targeting'], dtype=object)

In [278]:
ad_creatives_df.columns.values

array(['id', 'image_url', 'object_story_spec', 'title'], dtype=object)

In [279]:
df = df.join(ad_creatives_df, lsuffix='creative', rsuffix='id')

In [280]:
df.columns.values

array(['ad_id', 'ad_name', 'adset_name', 'campaign_name', 'card_adds',
       'cart_adds', 'checkouts', 'clicks', 'cpc', 'cpm', 'date_start',
       'date_stop', 'impressions', 'purchases', 'spend', 'view_contents',
       'creative', 'idcreative', 'status', 'targeting', 'idid',
       'image_url', 'object_story_spec', 'title'], dtype=object)

In [281]:
len(df)

143

In [282]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 24 columns):
ad_id                143 non-null object
ad_name              143 non-null object
adset_name           143 non-null object
campaign_name        143 non-null object
card_adds            55 non-null float64
cart_adds            143 non-null int64
checkouts            143 non-null int64
clicks               143 non-null int64
cpc                  143 non-null float64
cpm                  143 non-null float64
date_start           143 non-null object
date_stop            143 non-null object
impressions          143 non-null int64
purchases            143 non-null int64
spend                143 non-null float64
view_contents        143 non-null int64
creative             143 non-null object
idcreative           143 non-null object
status               143 non-null object
targeting            143 non-null object
idid                 143 non-null object
image_url            143 non-null obj

In [283]:
df.spend.sum()

8576.720000000001

In [284]:
df.impressions.sum()

932709

In [285]:
df.clicks.sum()

20139

In [286]:
df.view_contents.sum()

12056

In [287]:
df.checkouts.sum()

425

In [288]:
df.purchases.sum()

90

In [289]:
df.columns.values

array(['ad_id', 'ad_name', 'adset_name', 'campaign_name', 'card_adds',
       'cart_adds', 'checkouts', 'clicks', 'cpc', 'cpm', 'date_start',
       'date_stop', 'impressions', 'purchases', 'spend', 'view_contents',
       'creative', 'idcreative', 'status', 'targeting', 'idid',
       'image_url', 'object_story_spec', 'title'], dtype=object)

In [290]:
df.view_contents.sum()

12056

In [291]:
df.to_csv('last_week.csv')